## BestPictures:

1. Import the data from https://en.wikipedia.org/wiki/Academy_Award_for_Best_Picture
2. Explore the data and identify things that would be worth exploring
3. Extract data and Store in df and csv using BeautifulSoup


In [3]:
import pandas as pd
import numpy as np
import requests 
from bs4 import BeautifulSoup



In [4]:
#GLOBAL STATIC to prepend to ahref
wiki_home_url ='https://en.wikipedia.org'

## Import the data via call to wiki

In [5]:
resp = requests.get('https://en.wikipedia.org/wiki/Academy_Award_for_Best_Picture')
soup = BeautifulSoup(resp.text, 'html.parser')


In [6]:

#print(soup.find(id="Academy_Award_for_Best_Picture"))
#Find all wikitables with Movies(first11)
wikitables = soup.findAll('table', class_="wikitable",limit=11)
#all_decades = get_nom_movies(wikitables)

In [7]:
#soup.find('div', class="legend")
legend = soup.find('div', class_="legend")

In [19]:
#first_table(this block would be a best picture winner)
#trs = first_table[0].findAll('tr', style="background:#FAEB86")
#bp = trs[0].findAll('td')[0].findAll('a')
#bp[0].text


'Wings'

year has href title and text and contains movies which contain href title and text
##for each wikitable  the first th has a row span telling how many movies-1 and the tds are movie 
#and studio(has href title and text)
#dict will look like decade years movies 

In [8]:
#Returns a List of columns headers from the table headers
def find_columns(ths):
    columns = []
    for col in ths:
        columns.append(col.text.rstrip())
    return columns

#parses a table row if a movie row and returns a dict movie
def parse_movie(row,bp=False):
    movie = {}
    movie_url = ''
    movie_url_title = ''
    movie_name = ''

    if row.find('a'):
        movie_url = wiki_home_url + row.find('a').attrs['href']
        movie_url_title = row.find('a').attrs['title']
        movie_name = row.text.rstrip()
    else:
        movie_name = row.text.rstrip()
    
    movie = {'movie_url': movie_url , 'movie_url_title': movie_url_title, 'movie_name': movie_name, 
             'best_picture': bp}
    return movie



#parses a table row if a Studio row and returns a dict studio
def parse_studio(row):
    studio = {}
    studio_url = ''
    studio_url_title = ''
    studio_name = ''
    
    if row.find('a'):
        #studio_url = wiki_home_url + row.find('a').attrs['href']
        #studio_url_title = row.find('a').attrs['title']
        studio_name = row.text.rstrip()
    else:
        #studio_url = wiki_home_url +''
        #studio_url_title = row.text.rstrip()
        studio_name = row.text.rstrip()
    studio = {'studio_url': studio_url , 'studio_url_title': studio_url_title, 'studio_name': studio_name}    
    return studio



#decade has years has movies matched to a studio
#decade = {decade='',year:'', movies={}}
#the below loop lookins into a wikitable and returns a list of movies for that decade

#Get all nominated movies from WikiBPictures
#passing tds and trs where bold
#returns a list of movies
def parse_wikitable(wikitables):
    decades =[]
    movies = []
    
    year = ''
    numinyear = 0
    bplist = []
    headers= []
    #a wiki table holds decades of movies
    for wikitable in wikitables:#decade
        ##find trs
        #find trs style=FA  is best pict
        #Find Td
        trows= wikitable.findAll('tr')
        #bold_trs = wikitable.find('tr', style="background:#FAEB86")
        
        for num,trow in enumerate(trows):
            headers = find_columns(trow.findAll('th'))
            tdatas =trow.findAll('td')
            bp = False
            if trow.has_attr('style'):
                bp = True

            for num,tdata in enumerate(tdatas):#year
                
                movie={}
                if tdata.has_attr('rowspan'):
                    year = tdata.find('a').text.rstrip()

                    numinyear =int(tdata.attrs['rowspan'])-1
                    continue
                else:
                    movie = parse_movie(tdata,bp)

                    movie['year'] =  parseyear(year)
                movies.append(movie)
            decades.append(movies)

    return decades


    
    
#ToDo
def parseyear(year):
    str_year = ''
    if int(year[:4]) < 1934:
        #print(int(year[:4])+1)
        str_year = str(int(year[:4])+1)
    else:
        #print(year[:4])
        str_year = year[:4]
    return str_year



#decades list from 
#0=1920s 1=1930s, 2=1940s, 3=1950s
#4=1960 5=1970 6=1980 7=1990 8=2000
#9=2010s 10=2020
def get_movies_indecade(num):
    return all_decades[num]
    
    
    
#def get_movies_year(year):


In [156]:
#returns a list of best Picture winners
#trs = first_table[0].findAll('tr', style="background:#FAEB86")
#bp = trs[0].findAll('td')[0].findAll('a')
#bp[0].text
def best_picture_list(trs):
    bplist = []
    name = ''
    for boldrow in trs:
        #if boldrow.findAll('td')[0].has_attr('a'):
        name = boldrow.findAll('td')[0].findAll('a')[0]
        bplist.append(name.text)
    #print(bplist)   
    return bplist
    
def is_best_picture(name,bplist):
    if name in bplist:
        return True
    else:
        return False

In [9]:
#Find all wikitables with Movies(first11)
wikitables = soup.findAll('table', class_="wikitable",limit=11)
all_decades = parse_wikitable(wikitables)

In [10]:
len(all_decades)

674

In [28]:
def bps_indecade(num):
    decade = all_decades[num]
    movie={}
    movies=[]
    for movie in decade:
        if movie['best_picture'] == True:
            movie = {'movie_name':movie['movie_name'],'year':movie['year']}
        
        movies.append(movie)
    return movies


In [14]:
all_decades[0:2]

[[{'movie_url': 'https://en.wikipedia.org/wiki/Wings_(1927_film)',
   'movie_url_title': 'Wings (1927 film)',
   'movie_name': 'Wings',
   'best_picture': True,
   'year': '1928'},
  {'movie_url': 'https://en.wikipedia.org/wiki/Famous_Players-Lasky',
   'movie_url_title': 'Famous Players-Lasky',
   'movie_name': 'Famous Players-Lasky',
   'best_picture': True,
   'year': '1928'},
  {'movie_url': 'https://en.wikipedia.org/wiki/7th_Heaven_(1927_film)',
   'movie_url_title': '7th Heaven (1927 film)',
   'movie_name': '7th Heaven',
   'best_picture': False,
   'year': '1928'},
  {'movie_url': 'https://en.wikipedia.org/wiki/Fox_Film',
   'movie_url_title': 'Fox Film',
   'movie_name': 'Fox',
   'best_picture': False,
   'year': '1928'},
  {'movie_url': 'https://en.wikipedia.org/wiki/The_Racket_(1928_film)',
   'movie_url_title': 'The Racket (1928 film)',
   'movie_name': 'The Racket',
   'best_picture': True,
   'year': '1928'},
  {'movie_url': '',
   'movie_url_title': '',
   'movie_name':